# Moving plate onto Alpaqua magnet using CORE grippers

Information:
- PLR Recipe ID #1
- Machines used:
  - Hamilton STAR
- Non-PLR dependencies: None 
- tags: #resourcemovement #plateadapter #star
- Author: Camillo Moschner
- Version history: 0 (2025-10-21)

## Set Protocol Mode

In [ ]:
protocol_mode = "simulation" # "execution" or "simulation"

---
## Import Statements

### Machine & Visualizer

In [ ]:
%load_ext autoreload
%autoreload 2
       
import random 
import time

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.resources.hamilton import STARLetDeck
from pylabrobot.visualizer.visualizer import Visualizer

if protocol_mode == "execution":

    from pylabrobot.liquid_handling.backends import STARBackend

    backend = STARBackend()

elif protocol_mode == "simulation":

    from pylabrobot.liquid_handling.backends.hamilton.STAR_chatterbox import STARChatterboxBackend
    
    backend = STARChatterboxBackend()


### Required Resources

In [ ]:
from pylabrobot.resources import (
    MFX_CAR_L5_base , # MFX CARRIERS
    MFX_DWP_rackbased_module, Hamilton_MFX_plateholder_DWP_metal_tapped,
    Alpaqua_96_magnum_flx,
    Azenta4titudeFrameStar_96_wellplate_200ul_Vb,
)

## Instantiate Frontend & Connect to Machine

In [ ]:
# # # Frontend & Visualizer Setup # # #

lh = LiquidHandler(backend=backend, deck=STARLetDeck())

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

await lh.backend.disable_cover_control() # 😈

## Configure Deck Layout

In [ ]:
# Setup MFX Carrier for Magnetic Bead Resuspension

mfx_plateholder_dwp_1 = Hamilton_MFX_plateholder_DWP_metal_tapped(name=f"mfx_plateholder_tapped_dwp_0")

mfx_carrier_tapped_plate_holder_example = MFX_CAR_L5_base(
  name="mfx_carrier_tapped_plate_holder_example",
  modules={
      0: mfx_plateholder_dwp_1,
  }
)

mfx_carrier_tapped_plate_holder_example[0] = wash_plate = Azenta4titudeFrameStar_96_wellplate_200ul_Vb(name="wash_plate_0")

lh.deck.assign_child_resource(mfx_carrier_tapped_plate_holder_example, rails=1)

# Setup Magnet-carrying MFX Carrier

DWP_module_0 = MFX_DWP_rackbased_module(name=f"DWP_module_0")
magnet_0 = Alpaqua_96_magnum_flx(name=f"alpaqua_magnet_0")
DWP_module_0.assign_child_resource(magnet_0)

mfx_carrier_magnet_example = MFX_CAR_L5_base(
  name="mfx_carrier_magnet_example",
  modules={
      0: DWP_module_0,
  }
)
magnet_0.plate_z_offset = 0.62  # <===== PLATE-SPECIFIC !
# empirical: distance between bottom of Alpaqua magnet hole bottom to
# cavity_bottom of the well that is placed on top of it
# use ztouch_probing to measure both 

lh.deck.assign_child_resource(mfx_carrier_magnet_example, rails=8)


## Execution

### Move Plate Onto Magnet PlateAdapter

In [ ]:
plate_index = 0 # always design for throughput adaptivness ;)

plate_to_move = lh.deck.get_resource(f"wash_plate_{plate_index}")
move_target = lh.deck.get_resource(f"alpaqua_magnet_{plate_index}")


back_channel_idx = random.randint(1, 6)  # Reduce wear & tear on any single channel

if protocol_mode == "simulation":
    time.sleep(2)
    
await lh.move_plate(
    plate=plate_to_move,
    to=move_target,
    use_arm="core",
    channel_1=back_channel_idx,
    channel_2=back_channel_idx + 1,
    pickup_distance_from_top=6,
    core_grip_strength=40,
    return_core_gripper=False,
)

if protocol_mode == "execution":
    # "smart" command, will ask operator for input if it cannot find plate in move_target location
    # place into condition for simulation mode

    # (1) check transfer success, (2) push plate flush
    await lh.backend.core_check_resource_exists_at_location_center(
        location=plate_to_move.get_absolute_location(),
        resource=plate_to_move,
        gripper_y_margin=9,
        enable_recovery=True,
        audio_feedback=False,
    )

print(lh.backend.core_parked)
# >>> False # save time - keep CORE grippers on channels during magnetisation time

if protocol_mode == "simulation":
    time.sleep(2)

### Move Plate back onto tapped PlateHolder

In [ ]:
move_target = lh.deck.get_resource(f"mfx_plateholder_tapped_dwp_{plate_index}")

await lh.move_plate(
    plate=plate_to_move,
    to=move_target,
    use_arm="core",
    channel_1=back_channel_idx,
    channel_2=back_channel_idx + 1,
    pickup_distance_from_top=6,
    core_grip_strength=40,
    return_core_gripper=False,
)

if protocol_mode == "execution":

    await lh.backend.core_check_resource_exists_at_location_center(
        location=plate_to_move.get_absolute_location(),
        resource=plate_to_move,
        gripper_y_margin=9,
        enable_recovery=True,
        audio_feedback=False,
    )
    
await lh.backend.put_core()

print(lh.backend.core_parked)
# >>> True